<a href="https://colab.research.google.com/github/Glasiermedic/India-Health-Census/blob/master/India_Health_revist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
%matplotlib inline
import seaborn as sns


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree

from sklearn.model_selection import cross_val_score
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import warnings
# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd")

In [0]:
#@title
ind_cen = pd.read_csv('https://raw.githubusercontent.com/Glasiermedic/India-Health-Census/master/Key_indicator_districtwise.csv')

In [3]:
ind_cen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Columns: 644 entries, State_Name to ZZ_Sex_Ratio_At_Birth_Urban_Upper_Limit
dtypes: float64(614), int64(28), object(2)
memory usage: 1.4+ MB


In [0]:
ind_cen_prep = ind_cen.drop(['State_Name','State_District_Name'], axis=1)

In [0]:
col_new = []
for col in ind_cen_prep.columns:
  col_new.append(col[3:])
#print(col_new)  
ind_cen_prep.columns = col_new  

In [0]:
#@title
#a list of all the variables containing data about hypertension
hyp_all_accute = []
# a list of all the variables excluding the 18 related to hypertension
non_hyp = []

# sorts the variables that contain or encompass the reports of hypertension
for item in ind_cen_prep.columns:
  if 'Hypertension' in item:
    hyp_all_accute.append(item)
  elif 'Any_Type_Of_Acute' in item:
    hyp_all_accute.append(item)
  else:
    non_hyp.append(item)

In [7]:
ind_cen_var = ind_cen_prep.copy

ind_cen_var = ind_cen_prep[list(non_hyp)]

print ("Total Included",ind_cen_var.shape)

Total Included (284, 624)


In [0]:
Y = ind_cen_prep['Having_Diagnosed_For_Chronic_Illness_Per_100000_Population_Hypertension_Person_Total']

In [0]:
ind_cen_var = ind_cen_var.fillna(method='bfill')

In [10]:
np.where(ind_cen_var.isnull().sum()>0)

(array([277, 278]),)

In [11]:
null_var=(np.where(ind_cen_var.isnull().sum()>0))
null=null_var[0]
null_list = ((null[0]),(null[1]))

null_list

(277, 278)

In [12]:
for col in null_list:
  print (ind_cen_var.columns[col])
  ind_cen_var = ind_cen_var.drop([ind_cen_var.columns[col]], axis=1)
  
print ( ind_cen_var.shape)

Total_Fertility_Rate_Rural
Women_Aged_20_24_Reporting_Birth_Of_Order_2__Above_Total
(284, 622)


In [13]:
zer_var = (np.where(ind_cen_var.min()<= 0))
zer = zer_var[0]
zer_list = ((zer[0]),(zer[1]))

zer_list

(23, 56)

In [14]:
for col in zer_list:
  print (ind_cen_var.columns[col])
  ind_cen_var = ind_cen_var.drop([ind_cen_var.columns[col]], axis=1)
  
print ( ind_cen_var.shape)

Average_Household_Size_St_Urban
Marriages_Among_Males_Below_Legal_Age_21_Years_Total
(284, 620)


In [15]:
np.where(ind_cen_var.isnull().sum()>0)

(array([275]),)

In [16]:
ind_cen_var.columns[[275]]

Index(['Total_Fertility_Rate_Urban'], dtype='object')

In [0]:
ind_cen_var = ind_cen_var.drop('Total_Fertility_Rate_Urban',axis =1)

In [18]:
ind_cen_var.shape

(284, 619)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(ind_cen_var, Y, test_size=0.2, random_state=1)

In [20]:
X_train.shape

(227, 619)

In [0]:
forest = RandomForestRegressor(max_depth=7, random_state=12, n_estimators=100, max_features = 20)

In [0]:
cv_forest = cross_val_score(forest, X_train, y_train, cv=10)

In [23]:
cv_forest.mean()

0.7442376700645532

In [0]:
#params = {'max_depth': range(1, 30),
         #'n_estimators': range(50,600,50), 'max_features': range(5,100,10)}

#grid = GridSearchCV(RandomForestRegressor(), param_grid=params , cv=10)
#grid.fit(X_train, y_train)

In [25]:
clf = tree.DecisionTreeRegressor(random_state= 12, max_depth = 100, max_features = 400)
clf.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=100, max_features=400,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=12, splitter='best')

In [26]:
cv_tree = cross_val_score(clf, X_train, y_train, cv = 10)
cv_tree.mean()

0.5065356881456685

In [27]:
params2 = {'max_depth': range(1, 30),
          'max_features': range(150,250,10)}

grid = GridSearchCV(clf, param_grid=params2 , cv=10)
grid.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=DecisionTreeRegressor(criterion='mse', max_depth=100, max_features=400,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=12, splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'max_depth': range(1, 30), 'max_features': range(150, 250, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
print("the best score using Grid Search:")
grid.best_score_





the best score using Grid Search:


0.602172081416974

In [29]:
print("the best parameters found by Grid Search:")
grid.best_params_

the best parameters found by Grid Search:


{'max_depth': 7, 'max_features': 200}

In [30]:
clf = tree.DecisionTreeRegressor(random_state= 12, max_depth = 10, max_features = 200)
clf.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=200,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=12, splitter='best')

In [0]:
cv_tree2 = cross_val_score(clf, X_train, y_train, cv = 10)

In [32]:
cv_tree2.mean()

0.1806852960167677